# [arxiv:1709.05327](https://arxiv.org/pdf/1709.05327.pdf)

## ATLAS + CMS Charge asymmetry combination (7 and 8 TeV)

In [1]:
import pandas as pd
import numpy as np
from blue import Blue

## Inclusive $\sqrt{s} = 7$ TeV combination

Let's begin with the inclusive 7 TeV charge-asymmetry combination. The first step of performing the combination is to read in the measured values from each experiment and their associated uncertainties and correlations. To avoid having that information written out directly in this notebook, we have stored it in a CSV file. We read it in using `pandas` and display the data in the notebook. Note that the value of the correlation between the measured values is just set to 0 here, and isn't used.

In [2]:
df7 = pd.read_csv('data/charge_asym7.csv', index_col='Exp')
df7.T

Exp,ATLAS,CMS,Corr
Value,0.006,0.004,0.0
StatData,0.010,0.010,0.0
StatSim,0.002,0.002,0.0
DetMod,0.004,0.007,0.0
PU,0.002,0.000,0.0
SigMod,0.000,0.002,0.5
PDF,0.001,0.002,1.0
Multijet,0.000,0.001,0.0
Wjets,0.002,0.004,0.5
SpecPhysMod,0.000,0.000,0.0


The next step is to prepare this data for the combination. We do this by seperating the correlation information from the results and systematics, and removing the `Value` from the correlations column. The line below takes the 'Corr' row and all columns except the first one, which corresponds to the value Column.

In [3]:
corr7 = df7.loc['Corr'][1:]

Then we drop the Correlation column from the dataframe.

In [4]:
df7 = df7.drop('Corr')

Now we have the data and correlations in the correct format, we can create an instance of the `Blue` class.

In [5]:
blue7 = Blue(df7, corr7)

We can now query properties of this to get the combined result:

In [6]:
blue7.combined_result

0.0052848837209302328

The correlation between the two measurements `blue7.total_correlations` gives us the correlation matrix (which in this case will be a 2 by 2 matrix so we take one of the off-diagonal elements.

In [7]:
blue7.total_correlations[0, 1]

0.035062547474858388

We can also look at the combined uncertainties. `blue7.combined_uncertainties` returns a `dict` of uncertainty names and values but it can be handy to convert this into a `pandas.Series` so we will do that. We use `numpy.round` to display the uncertainties to three decimal places. The numbers for some of the uncertainties and the correlation between measurements are slightly different from what appears in the paper. I expect that this is because we set uncertainties listed in the paper as `<0.001` to `0`.

In [8]:
uncerts = pd.Series(blue7.combined_uncertainties)
np.round(uncerts, 3)

DetMod         0.004
GenMod         0.003
Multijet       0.000
PDF            0.001
PU             0.001
SigMod         0.001
SpecPhysMod    0.000
StatData       0.007
StatSim        0.001
Wjets          0.002
dtype: float64

We can compute the total systematic by summing all the uncertainties in quadrature except the statistical uncertainty.

In [9]:
systs = uncerts.drop('StatData')
total_syst = np.sqrt((systs**2).sum())
total_syst

0.0055643878472529723

We now have all the information to nicely print out the result.

In [10]:
print(f'A_C (7TeV) = {blue7.combined_result:.3f} '
      f'\pm {uncerts["StatData"]:.3f} (stat) '
      f'\pm {total_syst:.3f} (syst)')

A_C (7TeV) = 0.005 \pm 0.007 (stat) \pm 0.006 (syst)


We can look at the weight given to each measurement:

In [11]:
blue7.weights

array([ 0.64244186,  0.35755814])

and calculate the $\chi^2$ of the combination:

In [12]:
blue7.chi2_ndf

(0.011627906976744188, 1)

and the $p$-value of the combination.

In [13]:
from scipy.stats import chi2
chi2.sf(*blue7.chi2_ndf)

0.91412834520141983

## Inclusive $\sqrt{s} = 8$ TeV combination

We can then follow the same procedure to perform the 8 TeV combination.

In [14]:
df8 = pd.read_csv('data/charge_asym8.csv', index_col='Exp')
df8.T

Exp,ATLAS,CMS,Corr
Value,0.0090,0.0033,0.0
StatData,0.0044,0.0026,0.0
StatSim,0.0010,0.0015,0.0
Leptons,0.0003,0.0001,0.0
JER,0.0005,0.0004,0.0
btagging,0.0004,0.0007,0.0
MET,0.0002,0.0000,0.0
PU,0.0000,0.0003,0.0
uJES,0.0010,0.0004,0.0
pJES,0.0009,0.0010,0.5


In [15]:
corr8 = df8.loc['Corr'][1:]
df8 = df8.drop('Corr')

In [16]:
blue8 = Blue(df8, corr8)

In [17]:
blue8.combined_result

0.0055125806451612903

In [18]:
blue8.total_correlations[0, 1]

0.13214901382012048

In [19]:
uncerts8 = pd.Series(blue8.combined_uncertainties)
np.round(uncerts8[df8.columns[1:]], 4)

StatData         0.0023
StatSim          0.0010
Leptons          0.0001
JER              0.0003
btagging         0.0005
MET              0.0001
PU               0.0002
uJES             0.0005
pJES             0.0008
mJES             0.0003
cJES             0.0008
EvGen            0.0003
Hadronisation    0.0002
Scale            0.0012
PDF              0.0004
Lumi             0.0001
EWBkg            0.0003
Multijet         0.0011
Wjets            0.0001
Method           0.0001
dtype: float64

In [20]:
total_syst8 = np.sqrt((uncerts8.drop('StatData') ** 2).sum())
total_syst8

0.0024813674671195734

In [21]:
print(f'A_C (8TeV) = {blue8.combined_result:.4f} '
      f'\pm {uncerts8["StatData"]:.4f} (stat) '
      f'\pm {total_syst8:.4f} (syst)')

A_C (8TeV) = 0.0055 \pm 0.0023 (stat) \pm 0.0025 (syst)


In [22]:
blue8.weights

array([ 0.38817204,  0.61182796])

In [23]:
blue8.chi2_ndf

(0.87338709677419324, 1)

In [24]:
chi2.sf(*blue8.chi2_ndf)

0.35001932063705843